<a href="https://colab.research.google.com/github/adibayaseen/PPI-Inhibitors/blob/main/code/16April_MLP_normalized_proteinseqfeatures_readfromfile_with_binders_and_random_both_as_negative_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set the Runtime->Change Runtime Type to GPU**

# Protein 3d structure assessment with graph neural networks

In [1]:
#!rm -r Data
!rm -r PPI-Inhibitors
!pip install biopython
!git clone https://github.com/adibayaseen/PPI-Inhibitors
#!pip install py3Dmol

rm: cannot remove 'PPI-Inhibitors': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.1 MB/s eta 0:00:00
Cloning into 'PPI-Inhibitors'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 371 (delta 76), reused 95 (delta 48), pack-reused 221
Receiving objects: 100% (371/371), 9.60 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [2]:
!pip install rdkit
!conda install -c bioconda biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 16.1 MB/s eta 0:00:00
/bin/bash: conda: command not found


In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 16.3 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
class IPPI_MLP_Net(nn.Module):
    def __init__(self):
        super(IPPI_MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2328, 1024)#4096)2328
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 100)
        self.fc5 = nn.Linear( 100,1)
    def forward(self,PFeatures,LigandFeatures):
          PFeatures=torch.FloatTensor(PFeatures).cuda()
          Cfeatures=torch.FloatTensor(LigandFeatures).cuda()#Compound_Net(LigandFeatures)
          PC_Features=torch.hstack((PFeatures,Cfeatures))
          x = torch.tanh(self.fc1(PC_Features))
          #pdb.set_trace()
          x = torch.tanh(self.fc2(x) )
          x = torch.tanh(self.fc3(x) )
          x = self.fc5(x) 
          return x

In [ ]:
"""
Sequence based features
"""
from tqdm import tqdm as tqdm
import pandas as pd
import pickle
from sklearn.model_selection import GroupKFold
from sklearn.metrics import auc,roc_auc_score,roc_curve,precision_score,recall_score,average_precision_score,precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from torchmetrics.classification import BinaryHingeLoss
import pickle
import matplotlib.pyplot as plt

if __name__=="__main__":
    #drivepath='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
    #Load Features and github path
    githubpath='/content/PPI-Inhibitors/'
    Ubench5InterfaceandSeq_dict=pickle.load(open(githubpath+'Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
    Pos_seqandInterfaceF_dict=pickle.load(open(githubpath+'Features/Pos_seqandInterfaceF_dict.npy',"rb"))
    Complex_AllFeatures_dict=dict( list (Pos_seqandInterfaceF_dict.items())+list (Ubench5InterfaceandSeq_dict.items())) 
    CompoundFingerprintFeaturesDict=pickle.load(open(githubpath+'Features/Compound_Fingerprint_Features_Dict.npy',"rb"))
    Z = []; Yo = []; A = [];Yp=[];AUC_ROC_final=[];Precision_final=[];Recall_final=[];Avg_P_final=[];
    #Load File and preprocessing
    with open(githubpath+'Data/WriteAllexamplesRandomBindersIdsAll_24JAN.txt') as f:
    #with open(githubpath+'Data/WriteAllexamplesRandomBindersIdsAll_24JAN_Binary.txt') as f:
        D = f.readlines()
    Labels=[];Ligandnames=[];Complexs=[];TestPoscomplexes=[];Batchlosslist=[];
    for d in tqdm(D):
      if len(d.split())==4:
          TestPoscomp,Complexname,Ligandname,label = d.split()
      else:
          TestPoscomp,Complexname,Ligandname,label = d.split()[0],d.split()[1],(' ').join(d.split()[2:-1]),d.split()[-1]
      TestPoscomplexes.append(TestPoscomp),Ligandnames.append(Ligandname);Complexs.append(Complexname);Labels.append(float (label))
    #########Make dictionary, Rootcomplexname=(complexname,compoundname),label
    Allexamples=dict (zip(zip(TestPoscomplexes,zip(Complexs,Ligandnames)),Labels))
    #Group kfold
    Alldata=list (Allexamples.keys())
    KK=[k[0].split('_')[0] for k in Alldata]
    groups = pd.DataFrame(KK)
    gkf = GroupKFold(n_splits=len(set (KK)))
    ###########
    AlltestExamples=[];Externallabels=[];ExternalscoresLOCO=[];covid19_Externallabels=[];covid19_ExternalscoresLOCO=[];Y_score=[];Y_t=[];classratio_dict={};
    Complexs,Ligandnames, Labels=np.array(Complexs),np.array(Ligandnames),np.array(Labels)
    for trainindex, testindex in gkf.split(KK, KK, groups=groups):
        Alldata=np.array(Alldata)
        train,test=Alldata[trainindex],Alldata[testindex]
        Ctr=[];Ptr=[];y_train=[];Ctrname=[];Ptrname=[];Xtr=[];G=[];
        #Split train and test
        for t in train:
            Ctrname.append(t[1][1]);Ctr.append(CompoundFingerprintFeaturesDict[t[1][1]]);
            Ptrname.append( t[1][0]);Ptr.append( Complex_AllFeatures_dict[t[1][0]]);y_train.append(Allexamples[t[0],t[1]])
        #standarization
        Pscaler = StandardScaler().fit(Ptr)
        Cscaler = StandardScaler().fit(Ctr)
        Ptr,Ctr = Pscaler.transform(Ptr), Cscaler.transform(Ctr)
        Ptrdict=dict (zip(Ptrname,Ptr))
        Ctrdict=dict (zip (Ctrname,Ctr))
        #Xtr=Protein features concatenated with compound features
        Xtr=np.hstack((Ptr,Ctr))
        G=dict(zip(zip(Ptrname,Ctrname),Xtr))
        ###############
        IPPI_Net = IPPI_MLP_Net().cuda()
        #criterion = BinaryHingeLoss().cuda()
        criterion = nn.MSELoss().cuda()
        optimizer = optim.Adam(list (IPPI_Net.parameters()),lr=0.001,weight_decay=0.0)#0001)#0.69 for 1mer single layer#, weight_decay=0.01, betas=(0.9, 0.999))
        y_train=torch.FloatTensor( y_train).cuda()
        ################load classratio_dict based on rrot complex
        classratio_dict=pickle.load(open(githubpath+'Features/Classratio_dict.npy','rb'))
        for epoch in range(200):
          Batchloss=0
          for n in range(len(train)):#int (len(train)/10)):
            complexname,Ligandname=train[n][1]
            output=IPPI_Net(Ptrdict[complexname],Ctrdict[Ligandname])
            loss=criterion(output, y_train[n].reshape(1))
            if y_train[n]==1.0:
              #print(train[n][0],classratio_dict[train[n][0]])
              loss=classratio_dict[train[n][0]]*loss
            Batchloss=Batchloss+loss#.cpu().data.numpy()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if Batchloss<0.015:
              break;
          Batchlosslist.append(Batchloss.cpu().data.numpy()/len(train))#int (len(train)/10))#
          print("Epoch",epoch,"Batch loss",Batchloss.cpu().data.numpy()/len(train))
          if (Batchloss.cpu().data.numpy()/len(train))<0.015:
              break;
        ######
        """
        plt.figure()
        plt.plot(Batchlosslist)
        plt.title('batchloss');plt.xlabel('epochs');plt.ylabel('loss');plt.grid();plt.legend();plt.show();
        """
        #Testing
        Ctt=[];Ptt=[];y_test=[];Cttname=[];Pttname=[];
        for t in test:
            Cttname.append(t[1][1]);Ctt.append(CompoundFingerprintFeaturesDict[t[1][1]]);
            Pttname.append( t[1][0]);Ptt.append( Complex_AllFeatures_dict[t[1][0]]);y_test.append(Allexamples[t[0],t[1]])
        Ptt,Ctt = Pscaler.transform(Ptt), Cscaler.transform(Ctt)
        Pttdict=dict (zip(Pttname,Ptt))
        Cttdict=dict (zip (Cttname,Ctt))
        for nt in range(len(test)):
          complexname,Ligandname =test[nt][1]
          test_score=IPPI_Net(Pttdict[complexname],Cttdict[Ligandname])
          Y_score.extend(test_score.cpu().data.numpy())
          Y_t.append(y_test[nt])#.cpu().data.numpy())
        ################
        Auc = roc_auc_score(Y_t, Y_score)
        average_P_score=average_precision_score(Y_t, Y_score)
        print(test[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3))
        AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
        Z.extend(list(Y_score));Yo.extend(list(Y_t))#;Yp.extend(list(yp))
        #save complexwise class ratio
        """
        y_test=np.array(y_test)#.cpu().data.numpy())
        classratio=round(len(y_test[y_test==0.])/len(y_test[y_test==1.]),2)
        classratio_dict[test[0][0]]=classratio
        print(classratio,test[0][0])
        """
        """
        #/content/PPI-Inhibitors/Data/External data/2dyh_all.txt#githubpath+'/Data/External data/2dyh_all.txt
        #RFPP_all=PredictRFPPfromFile(githubpath,githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
        External_score,External_labels=PredictScorefromFileSVM(githubpath,githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
        ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
        External_Auc= roc_auc_score(External_labels, External_score)
        External_AP=average_precision_score(External_labels, External_score)
        print("External_Auc,PR,RFPP",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
        #########
        #Covid19_RFPP_all=PredictRFPPfromFile(githubpath,githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
        Covid19_External_score,Covid19_External_labels=PredictScorefromFileSVM(githubpath,githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
        covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
        Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
        Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
        print("Covid19_External_Auc,PR,RFPP",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")
    #####
    External_fpr, External_tpr, External_thresholds = roc_curve(Externallabels,ExternalscoresLOCO)
    External_Auc = roc_auc_score(Externallabels,ExternalscoresLOCO)
    External_Auc=(External_Auc).round(2)
    fig = plt.figure()
    plt.plot(External_fpr, External_tpr,color='k',marker='d',label='External_Auc:{: .2f}'.format(External_Auc))
    plt.title('AUCROC External');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
    fig .savefig(path+"AUCROC External SVM PPI Inhibitors Random and Binders combine Negative.pdf", bbox_inches='tight')
    ########
    covid19_External_fpr, covid19_External_tpr, covid19_External_thresholds = roc_curve(covid19_Externallabels,covid19_ExternalscoresLOCO)
    covid19_External_Auc = roc_auc_score(covid19_Externallabels,covid19_ExternalscoresLOCO)
    covid19_External_Auc=(covid19_External_Auc).round(2)
    fig = plt.figure()
    plt.plot(covid19_External_fpr, covid19_External_tpr,color='k',marker='d',label='covid19_External_Auc:{: .2f}'.format(covid19_External_Auc))
    plt.title('covid19 AUCROC External');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
    fig .savefig(path+"AUCROC covid19 External SVM PPI Inhibitors Random and Binders combine Negative.pdf", bbox_inches='tight')
    ########
    """
    ###
    #pickle.dump(classratio_dict,open('/content/drive/MyDrive/GNN-PPI-Inhibitor'+'/Classratio_dict.npy','wb'))
    fpr, tpr, thresholds = roc_curve(Yo, Z)  
    Auc = roc_auc_score(Yo, Z)
    Auc=(Auc).round(2)
    # calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(Yo, Z)
    aucpr=auc(recall,precision)
    aucpr=(aucpr).round(2)
    #####
    Yo=np.array(Yo)
    print("AucROC and aucpr\n",Auc,"\n",aucpr,"\ntotal P:N ration 1:",int (np.sum([Yo==-1.0])/np.sum([Yo==1.0])),"\n")
    ######
    path=githubpath
    fig = plt.figure()
    plt.plot(fpr,tpr,color='k',marker='d',label='AUC:{: .2f}'.format(Auc))
    plt.title('AUCROC');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();
    fig .savefig(path+"AUCROC MLP-seqFeatures PPI Inhibitors Random and Binders combine Negative.pdf", bbox_inches='tight')
    ###
    precision, recall, thresholds = precision_recall_curve(Yo, Z)
    aucpr=average_precision_score (Yo, Z)  
    ######
    fig = plt.figure()
    plt.plot(recall,precision,color='m',marker=',',label='AUC-PR:{: .2f}'.format(aucpr))
    plt.title('AUC-PR');plt.xlabel('Recall');plt.ylabel('Precision');plt.grid();plt.legend();plt.show();
    fig .savefig(path+"AUC-PR MLP-seqFeatures PPI Inhibitors  Random and Binders combine.pdf", bbox_inches='tight')
    print("Final average over all folds,Leave one complex out",np.average(AUC_ROC_final).round(4),'±',np.std( AUC_ROC_final).round(4),np.average(Avg_P_final).round(4),'±',np.std( Avg_P_final).round(4))

100%|██████████| 15695/15695 [00:00<00:00, 625417.80it/s]
<ipython-input-5-d0babf7c3047>:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Alldata=np.array(Alldata)


Epoch 0 Batch loss 0.8292280483079721
Epoch 1 Batch loss 0.6926794137142638
Epoch 2 Batch loss 0.6366149796021621
Epoch 3 Batch loss 0.6020523323022339
Epoch 4 Batch loss 0.54450979544523
Epoch 5 Batch loss 0.5648442903665847
Epoch 6 Batch loss 0.4842808361192233


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
classratio_dict

In [ ]:
pickle.dump(classratio_dict,open(githubpath+'Features/Classratio_dict.npy','wb'))

In [ ]:
y_test=np.array(y_test)#.cpu().data.numpy())
classratio=len(y_test[y_test==0.])/len(y_test[y_test==1.])
print(classratio,test[0][0])
classratio_dict={}
classratio_dict[test[0][0]]=classratio


In [ ]:
y_train=np.array(y_train.cpu().data.numpy())

In [ ]:
y_test

In [ ]:
train[0][0]

In [ ]:
train[n][0]

In [ ]:
complexname

In [ ]:
len(y_train[y_train==1.])

In [ ]:
classratio=len(y_test[y_test==0.])/len(y_test[y_test==1.])
classratio_dict={}
classratio_dict[test[0][0]]=classratio
print(classratio,test[0[0]])

In [ ]:
#y_train=np.array(y_train.cpu().data.numpy())
np.sum(np.where(y_train[y_train==1.]))

In [ ]:
plt.hist(y_train)

In [ ]:
KK[-100:]

In [ ]:
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  print (Ligandname,CompoundFingerprintFeaturesDict[Ligandname])
  #complexname,Complex_AllFeatures_dict[complexname],

In [ ]:
print(test[0][0],"\t",round (Auc,3),"\t",round (average_P_score,3))

In [ ]:
t[0][0]

In [ ]:
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  print (Cttdict[Ligandname])

In [ ]:
print (Ptt[0:11])

In [ ]:
y_test=[];Y_t=[];Y_score=[];
for n in range(len(test)):
    y_test.append(Allexamples[test[n][0],test[n][1]])
#y_test=torch.FloatTensor( y_test).cuda()
for nt in range(len(test)):
  complexname,Ligandname =test[nt][1]
  test_score=IPPI_Net(CompoundNet,Complex_AllFeatures_dict[complexname],CompoundFingerprintFeaturesDict[Ligandname])
  Y_score.extend(test_score.cpu().data.numpy())
  Y_t.append(y_test[nt])#.cpu().data.numpy())
################
Auc = roc_auc_score(Y_t, Y_score)
average_P_score=average_precision_score(Y_t, Y_score)
#        print("\nfold Auc:",Auc,"average_P_score=",average_P_score ,"P",len(testPos),"N",len(testNeg))
print(t[0],"\t",round (Auc,3),"\t",round (average_P_score,3))# ,"\t",len(),"\t",len(testNegcomp),"\t",round (len(testNegcomp) /len(testPoscomp),1))
#        pdb.set_trace()
AUC_ROC_final.append(Auc);Avg_P_final.append(average_P_score);
Z.extend(list(z));Yo.extend(list(y_test))#;Yp.extend(list(yp))
#######################3
#/content/PPI-Inhibitors/Data/External data/2dyh_all.txt#githubpath+'/Data/External data/2dyh_all.txt
#RFPP_all=PredictRFPPfromFile(githubpath,githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
External_score,External_labels=PredictScorefromFileSVM(githubpath,githubpath+'/Data/External data/2dyh_all.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
ExternalscoresLOCO.extend(External_score);Externallabels.extend(External_labels)
External_Auc= roc_auc_score(External_labels, External_score)
External_AP=average_precision_score(External_labels, External_score)
print("External_Auc,PR,RFPP",round (External_Auc,3), round (External_AP,3))#,RFPP_all,"\n")
#########
#Covid19_RFPP_all=PredictRFPPfromFile(githubpath,githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ctr,Ptr)
Covid19_External_score,Covid19_External_labels=PredictScorefromFileSVM(githubpath,githubpath+'/Data/External data/HansonACE2hits.txt',githubpath+'/Data/External data/pdb/',clf,Ptr,Ctr,Pscaler, Cscaler)#(filename,Pdbloc,trainedModel_SVM,Ptr,Ctr)
covid19_Externallabels.extend(Covid19_External_labels);covid19_ExternalscoresLOCO.extend(Covid19_External_score)
Covid19_External_Auc= roc_auc_score(Covid19_External_labels, Covid19_External_score)
Covid19_External_AP=average_precision_score(Covid19_External_labels, Covid19_External_score)
print("Covid19_External_Auc,PR,RFPP",round (Covid19_External_Auc,3), round (Covid19_External_AP,3))#,Covid19_RFPP_all,"\n")

In [ ]:
plt.hist(Y_score)

In [ ]:
Y_t

In [ ]:
y_test[nt]

In [ ]:
y_test

In [ ]:
plt.figure()
plt.plot(Batchlosslist)
plt.title('batchloss');plt.xlabel('epochs');plt.ylabel('loss');plt.grid();plt.legend();plt.show();

In [ ]:
output

In [ ]:
Complex_AllFeatures_dict[complexname].shape

In [ ]:
Allexamples[train[n][0],train[n][1]]

In [ ]:
train[n]

In [ ]:
train[n][0],train[n][1]

In [ ]:
plt.plot(External_score)
plt.figure()
plt.plot(External_labels)

In [ ]:
External_fpr, External_tpr, External_thresholds = roc_curve(Externallabels,ExternalscoresLOCO)
External_Auc = roc_auc_score(Externallabels,ExternalscoresLOCO)
External_Auc=(External_Auc).round(2)
fig = plt.figure()
plt.plot(External_fpr, External_tpr,color='k',marker='d',label='External_Auc:{: .2f}'.format(External_Auc))
plt.title('AUCROC External');plt.xlabel('FPR');plt.ylabel('TPR');plt.grid();plt.legend();plt.show();

In [ ]:
ExternalscoresLOCO